<a href="https://colab.research.google.com/github/micah-shull/LLMs/blob/main/LLM_055_RAG_CahsFlow4Cast_Chatbot_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Import & Install Dependencies



In [1]:
!pip install -q gradio transformers dotenv huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.5 MB/s eta 0:00:00


## BASELINE CHAT MODEL

In [2]:
from huggingface_hub import login
from transformers import pipeline
import gradio as gr

# Load the model
chatbot = pipeline("text2text-generation", model="google/flan-t5-large")

# Store chat history for debugging
chat_log = []

def respond(message):
    prompt = f"""Q: What are economic indicators?
A: Economic indicators like inflation, consumer confidence, and employment rates help you anticipate changes in customer spending.

Q: How can I improve cash flow?
A: Track expenses weekly, forecast sales 30 days out, and reduce inventory waste.

Q: How do forecasting models work?
A: Traditional tools like Excel rely on past data averages. More advanced models use machine learning and local trends to improve forecast accuracy.

Q: {message}
A:"""

    response = chatbot(prompt, max_new_tokens=250, temperature=0.7)[0]["generated_text"].strip()
    chat_log.append((message, response))

    print("\n" + "="*60 + "\n🧾 Chat Log\n" + "="*60)
    for i, (q, a) in enumerate(chat_log, 1):
        print(f"\n{i}. Q: {q}\n   A: {a}\n{'-'*50}")

    return response

# Create the Gradio interface
demo = gr.Interface(
    fn=respond,
    inputs=gr.Textbox(lines=2, placeholder="Ask about forecasting, economics, or small business strategy..."),
    outputs=gr.Textbox(lines=6, label="Assistant Response"),
    title="📊 Small Business Forecasting Assistant"
)

# Launch the chatbot
demo.launch(share=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cpu


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d9ef02fb6da64e6023.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


#### Chat Logs

In [3]:
print("🧾 Chat Log\n" + "=" * 40)
for i, (user_msg, bot_msg) in enumerate(chat_log, start=1):
    print(f"\n🧑‍💼 User {i}: {user_msg}")
    print(f"  🤖 Bot  {i}: {bot_msg}")
    # print("-" * 40)

🧾 Chat Log

🧑‍💼 User 1: Hi can you teach me how economic indicators affect my small business?
  🤖 Bot  1: Economic indicators like inflation, consumer confidence, and employment rates help you anticipate changes in customer spending.

🧑‍💼 User 2: Great, how can i use those to improve my cash flow forecasting?

  🤖 Bot  2: Track expenses weekly, forecast sales 30 days out, and reduce inventory waste.

🧑‍💼 User 3: How can i combined economic indicators with my weekly sales to increase the accuracy of my cash flow forecasting?

  🤖 Bot  3: Use a spreadsheet to track your sales and economic indicators.


#### Memory Cleanup (Before Switching Models)

In [5]:
import gc
import torch

# Clear GPU memory
torch.cuda.empty_cache()
gc.collect()

del chatbot

# RAG

### 📄 Purpose:
> This code enables semantic retrieval of relevant blog content using FAISS and a sentence transformer. It powers the RAG system by embedding the user’s question, searching the FAISS index for similar blog chunks, and returning a combined context to inform the chatbot’s answer.

### 🔑 Key Concepts:
- **FAISS Index**: Stores precomputed vector embeddings of blog content.
- **Embedding Model**: Converts user questions into comparable vectors.
- **Semantic Search**: Finds conceptually relevant blog chunks, even if the wording is different.
- **Context Assembly**: Combines top chunks to provide a reference for answer generation.

---

## 🔍 Step-by-Step Explanation

### 1. **Reload the FAISS index and metadata**
```python
index = faiss.read_index("/content/blog_index.faiss")
df_clean_chunks = pd.read_csv("/content/blog_metadata.csv")
```
- `faiss.read_index(...)`: Loads the vector search index (stored embeddings of your blog chunks).
- `pd.read_csv(...)`: Loads the metadata for those chunks — including the actual blog text, titles, and filenames.

✅ These two steps restore everything you need for semantic search.

---

### 2. **Load the Sentence Embedding Model**
```python
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
```
- Loads the same embedding model you used **when chunking and indexing your blog**.
- This ensures that any new input (e.g. user questions) is embedded into the **same vector space**.

> 🔁 If the question and chunks aren't in the same space, the similarity search won't work correctly.

---

### 3. **Define a Retrieval Function**
```python
def retrieve_context(query, k=5):
    # Step 1: Embed the user query
    query_embedding = embedding_model.encode([query])[0].astype("float32")
```
- Embeds the user's natural-language question into a numeric vector (`query_embedding`) for similarity comparison.

```python
    # Step 2: Perform FAISS search
    D, I = index.search(np.array([query_embedding]), k)
```
- `index.search(...)` returns the `k` most similar chunks:
  - `D` is the distance (lower = more similar)
  - `I` is the index of the matching rows in `df_clean_chunks`

```python
    # Step 3: Retrieve matching blog chunks
    retrieved_chunks = []
    for idx in I[0]:
        chunk = df_clean_chunks.iloc[idx]["text"]
        retrieved_chunks.append(chunk)
```
- Loops through the `k` top matching indices and grabs the corresponding text chunks from your DataFrame.

```python
    # Step 4: Combine into a single reference document
    context = "\n\n".join(retrieved_chunks)
    return context
```
- Combines the relevant chunks into one string (`context`) that can be added to your chatbot prompt as background information.








### install dependencies

In [13]:
!pip install -q faiss-cpu

### load chunks and meadata

In [15]:
import faiss
import pandas as pd
import numpy as np

# Reload FAISS index and metadata
index = faiss.read_index("/content/blog_index.faiss")
df_clean_chunks = pd.read_csv("/content/blog_metadata.csv")

print("✅ FAISS index and metadata reloaded.")

✅ FAISS index and metadata reloaded.


### load model def retreival function

In [7]:
from sentence_transformers import SentenceTransformer

# Load the same embedding model used to create the FAISS index
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

def retrieve_context(query, k=5):
    # Embed the user question
    query_embedding = embedding_model.encode([query])[0].astype("float32")

    # Perform FAISS search
    D, I = index.search(np.array([query_embedding]), k)

    # Retrieve corresponding chunks
    retrieved_chunks = []
    for idx in I[0]:
        chunk = df_clean_chunks.iloc[idx]["text"]
        retrieved_chunks.append(chunk)

    # Combine into a reference context for the chatbot
    context = "\n\n".join(retrieved_chunks)
    return context

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### test retreival function

In [16]:
query = "How do I improve my sales forecasting accuracy?"
context = retrieve_context(query)
print(context)


📋 Store 5 – Forecasting Accuracy Summary Model MAE ($) MAPE (%) RMSE ($) Excel 1,275 10.70% 2,523 Machine Learning 518 4.86% 699 ML forecasts cut Store 5’s average error by more than 50% — delivering more accurate predictions and helping the team react faster when sales patterns shift. We evaluate forecast performance using three key metrics: MAE (Mean Absolute Error): This tells us how far off the forecast was each day, on average — in dollars. Lower is better. MAPE (Mean Absolute % Error): A percentage-based measure that shows how inaccurate the forecast was, relative to actual sales.

📋 Store 17 – Forecasting Accuracy Summary Model MAE ($) MAPE (%) RMSE ($) Excel 5,958 44.30% 6,928 Machine Learning 2,190 16.10% 2,530 ML forecasting reduced Store 17’s error by over 60%, offering sharper accuracy and clearer early signals for business planning. We evaluate forecast performance using three key metrics: MAE (Mean Absolute Error): This tells us how far off the forecast was each day, on a

#### Remove Widgets

In [ ]:
import json
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

notebook_path = "/content/drive/My Drive/LLM/LLM_055_RAG_CahsFlow4Cast_Chatbot_RAG.ipynb"

# Load the notebook JSON
with open(notebook_path, 'r', encoding='utf-8') as f:
    nb = json.load(f)

# 1. Remove widgets from notebook-level metadata
if "widgets" in nb.get("metadata", {}):
    del nb["metadata"]["widgets"]
    print("✅ Removed notebook-level 'widgets' metadata.")

# 2. Remove widgets from each cell's metadata
for i, cell in enumerate(nb.get("cells", [])):
    if "metadata" in cell and "widgets" in cell["metadata"]:
        del cell["metadata"]["widgets"]
        print(f"✅ Removed 'widgets' from cell {i}")

# Save the cleaned notebook
with open(notebook_path, 'w', encoding='utf-8') as f:
    json.dump(nb, f, indent=2)

print("✅ Notebook deeply cleaned. Try uploading to GitHub again.")